In [119]:
import math

# Ultimate PWM dimmer
- Low quiescent current voltage regulator: AP7381
  - 2.5uA quiescent current
- ATTINY25V
  - https://ww1.microchip.com/downloads/en/DeviceDoc/Atmel-2586-AVR-8-bit-Microcontroller-ATtiny25-ATtiny45-ATtiny85_Datasheet.pdf
  - 128kHz internal low speed oscillator (datasheet 6.2.4)
  - Timer/Counter0 times UI intervals at about 64Hz (base clock with prescaler 8 and counting up to 0xff)
  - Fall back to IDLE mode as often as possible (all work done in interrupts)


## States
- *On*
  - Output PWM is running (or 100% on)
  - Handle user input from the button
  - Mostly in IDLE state
  - Must switch to *Off* state during power loss
- *Off*
  - State is already saved in EEPROM
  - Output is off
  - MCU in SLEEP state, waiting for external interrupt from the button
- *No power*

## Transistor losses calculation

In [120]:
Imax = 5

In [121]:
Umin = 5
Umax = 24

Timer/Counter1 running from internal PLL with LSM bit set (32Mhz), prescaler 8, OCR1C = 0xA7

In [122]:
f_pwm = 32*2**20 / 8 / 0xa7
f_pwm

25115.59281437126

In [143]:
Igate_max = 70e-3

In [144]:
def transistor_losses(Rdson, Qgsgd):
    conduction_loss = Imax**2 * Rdson
    switching_loss = Umax * Imax * f_pwm * Qgsgd / Igate_max
    return {"conduction": conduction_loss, "switching": switching_loss, "total": conduction_loss + switching_loss}

### PMV20XNER

In [145]:
transistor_losses(Rdson=25e-3, Qgsgd=3.3e-9)

{'conduction': 0.625,
 'switching': 0.14208249649272883,
 'total': 0.7670824964927289}

### ECH8697R
Not usable -- too low $V_{DS Max}$

In [146]:
transistor_losses(Rdson=11.6e-3, Qgsgd=2e-9)

{'conduction': 0.29,
 'switching': 0.08611060393498716,
 'total': 0.37611060393498713}

### AO4752
https://www.digikey.cz/product-detail/en/alpha-omega-semiconductor-inc/AO4752/785-1597-1-ND/3712546

In [147]:
transistor_losses(Rdson=15.5e-3, Qgsgd=4.6e-9)

{'conduction': 0.3875,
 'switching': 0.19805438905047046,
 'total': 0.5855543890504704}

### IRL6342PbF

In [148]:
transistor_losses(Rdson=14.6e-3, Qgsgd=5.2e-9)

{'conduction': 0.365,
 'switching': 0.22388757023096664,
 'total': 0.5888875702309666}

### IRLR8726PbF

In [149]:
transistor_losses(Rdson=8e-3, Qgsgd=1.9e-9+5.7e-9)

{'conduction': 0.2,
 'switching': 0.3272202949529512,
 'total': 0.5272202949529512}

## Transistor gate resistor

In [161]:
r_pin = 0.44/20e-3
r_gate_internal = 2  # For IRL6342Pbf
i_pin = 40e-3
vcc = 5 * 1.02 - 0.4

In [162]:
r_gate = vcc / (2 * i_pin) - r_pin / 2 - r_gate_internal
r_gate

45.74999999999999

## Voltage sensing

AN2447 !!
    - Use Vcc as ADC reference
    - use V_ref as measured value
    - then Vcc = 1024 * V_ref / output

In [106]:
v_ref = 1.1 # AVR's internal votage reference
v_ref_min = 1.0
v_ref_max = 1.2
v_trigger = 3.7 # We want the power down to trigger (approximately) at Vcc = 3.7V
v_min = 2 # AVR can safely do EEPROM writes until 2V
r_equiv_saving = 30e3 / 2  # Calculated from active power consumption at 128khz and around 4.5V, 2x safety margin
r_equiv_running = 42e3 / (2 * (1 + 0.8 + 0.25)) # Calculated from idle power consumption at 128kHz and around 4V, with Timer1 and ADC enabled, 2x safety margin
f_cpu_min = 114e3
adc_sampling_time = 1 * 13 / f_cpu_min
adc_accuracy = 2

In [107]:
trigger_adc_output = round(1024 * v_ref / v_trigger)
trigger_adc_output

304

In [108]:
min_v_trigger = 1024 * v_ref_min / (trigger_adc_output + adc_accuracy)
min_v_trigger

3.34640522875817

In [109]:
max_v_trigger = 1024 * v_ref_max / (trigger_adc_output - adc_accuracy)
max_v_trigger

4.0688741721854305

In [110]:
def capacitor_discharge_time(v0, v1, r, c):
    return -r * c * math.log(v1 / v0)
def capacitor_discharge_voltage(v0, r, c, t):
    return v0 * math.exp(-t / (r * c))

In [111]:
def poweroff_runtime(c):
    voltage_when_detected = capacitor_discharge_voltage(min_v_trigger, r_equiv_running, c, adc_sampling_time)
    runtime = capacitor_discharge_time(voltage_when_detected, v_min, r_equiv_saving, c)
    return {"runtime": runtime,
            "voltage_when_detected": voltage_when_detected}

In [112]:
poweroff_runtime(2.2e-6 * 0.9)

{'runtime': 0.01512078388581299, 'voltage_when_detected': 3.3276437904558582}